In [15]:
import nibabel as nib
import numpy as np
from scipy import stats
import mvlearn
from mvlearn import embed
import os
import pandas as pd
import os
import re


def load_dts(dtseries_path):
    dtseries = nib.load(dtseries_path)
    
    L_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_LEFT']
    R_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_RIGHT']
    offset_count = [L_model[0].index_offset, L_model[0].index_count,
                    R_model[0].index_offset, R_model[0].index_count]
    values = dtseries.get_fdata()
    values = values[0:,np.append(np.arange(offset_count[0],offset_count[0]+offset_count[1]),np.arange(offset_count[2],offset_count[2]+offset_count[3]))]
    return stats.zscore(values, axis=0, nan_policy='omit')


def concat_dts(input_dir, output_dir):
    sbj_dirs = next(os.walk(input_dir))[1]
    sbj_IDs = []
    for sbj in sbj_dirs:
        print('searching folder:' + sbj)
        sbj_tmp = input_dir + sbj
        
        files = []
        try:
            for root, dirnames, filenames in os.walk(sbj_tmp):
                for file in filenames:
                    files.append(os.path.join(root, file))

            dts1_lr_path = next((x for x in files if x.endswith('rfMRI_REST1_LR_Atlas_hp2000_clean.dtseries.nii')), None)
            dts1_rl_path = next((x for x in files if x.endswith('rfMRI_REST1_RL_Atlas_hp2000_clean.dtseries.nii')), None)
            #dts2_lr = next((x for x in files if x.endswith('rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii')), None)
            #dts2_rl = next((x for x in files if x.endswith('rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii')), None)
            dts1_lr = load_dts(dts1_lr_path)
            dts1_rl = load_dts(dts1_lr_path)
            dtss_all = np.concatenate([dts1_lr, dts1_rl]).T# , load_dts(dts2_lr) , load_dts(dts2_rl)]).T
            
            np.save(output_dir + sbj + '_rfMRI_REST1_LR_RL.npy' , dtss_all)
            sbj_IDs.append(sbj)
            del files , dtss_all , dts1_lr , dts1_rl# , dts2_lr , dts2_rl
        except:
             print('Something went wrong')
    np.save(output_dir+'subject_IDs.npy', sbj_IDs)
        

concat_dts("/home/fralberti/Data/HCP_func/","/home/fralberti/Data/Concatenated_dtseries/")

In [4]:
sbj_IDs = np.load('/home/fralberti/Data/Concatenated_dtseries/subject_IDs.npy')
dtseries_list = [np.load('/home/fralberti/Data/Concatenated_dtseries/%s_rfMRI_REST1_LR_RL.npy' % x) for x in sbj_IDs]

gcca = embed.GCCA(n_components=4)
res = gcca.fit_transform(dtseries_list)
del dtseries_list

for i,sbj in enumerate(sbj_IDs):
    np.save('/home/fralberti/Data/Gradient_1/%s_grad1.npy' % sbj,res[i,:,0])
    
del res

In [5]:
def mk_grad1_dscalar(subject_ID, grad1_npy, template_dscalar, output_dir):

    data_length = template_dscalar.shape[1]
    data = np.zeros([1,data_length])
    data[0,0:grad1_npy.shape[0]] = grad1_npy

    ax0 = nib.cifti2.cifti2_axes.ScalarAxis(['Gradient 1'])
    ax1 = template_dscalar.header.get_axis(1)
    new_img = nib.Cifti2Image(data, header=[ax0, ax1],nifti_header=template_dscalar.nifti_header)
    new_img.update_headers()

    new_img.to_filename("/home/fralberti/Data/Gradient_1/%s_grad1.dscalar.nii" % subject_ID)
    del template_dscalar
    
    

for root, dirs, files in os.walk("/home/fralberti/Data/Gradient_1/"):
    npy_files = list(filter(lambda x: re.match('.*\.npy$', x), files))
    for file in npy_files:
        subject_ID = file.split("_")[0]
        path_to_data = os.path.join(root, file)
        grad1_npy = np.load(path_to_data)
        template_dscalar = nib.load("/home/fralberti/Data/HCP_zone_prim/%s/%s_tfMRI_LANGUAGE_level2_hp200_s12.dscalar.nii" % (subject_ID,subject_ID))
        output_dir = "/home/fralberti/Data/Gradient_1/"
        
        mk_grad1_dscalar(subject_ID, grad1_npy, template_dscalar, output_dir)



['100610_grad1.npy', '101006_grad1.npy', '100408_grad1.npy']


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


100610


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


101006


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


100408
[]
